# Today we will be creating a prime number generator, and pulling useful data from the generated values.

Let's start by modelling our data.

In [2]:
struct Primes {
    data: Vec<i64>,
}

We should implement a method for generating the prime numbers.

In [3]:
impl Primes {
    fn new(size: usize) -> Self {
        let mut primes = vec![2];
        for count in (3..).step_by(2) {
            if primes.len() >= size {
                break;
            }

            let mut is_prime = true;

            for &p in &primes {
                if count % p == 0 {
                    is_prime = false;
                    break;
                }
                if p * p >= count {
                    break;
                }
            }

            if is_prime {
                primes.push(count);
            }
        }

        Self { data: primes }
    }
}

We can now easily generate a vector of prime numbers with a given size. Here is how that works.

In [4]:
println!("{:?}", Primes::new(25).data);

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


Now that we have our data, let's define our next structure. We should create a structure containing all the data we want to extract.

In [5]:
struct DisplayValues {
    id: usize,
    value: i64,
    difference: i64,
    parity: Parity,
    mod_4: i64,
}

#[derive(Debug)]
enum Parity {
    Even,
    Odd,
}

use Parity::*;

Now we need a method for converting our `Prime` type into a vector of `DisplayValues`.

In [6]:
impl Primes {
    fn to_dv(self) -> Vec<DisplayValues> {
        let mut last = 2;
        self.data
            .into_iter()
            .enumerate()
            .map(|(id, p)| DisplayValues {
                id,
                value: p,
                difference: {
                    let r = p - last;
                    last = p;
                    r
                },
                parity: if p & 1 == 0 { Even } else { Odd },
                mod_4: p % 4,
            })
            .collect()
    }
}

Implementing `to_dv()` allows us to do the following.

In [7]:
let display_values = Primes::new(3).to_dv();

Now that we've modeled what values will be present in our csv file, we should implement a method for creating a csv formatted string from DisplayValues. We could do that for just DisplayValues, but this kind of implementation might do better as a trait. Let's define that.

In [8]:
trait ToCSV {
    const KEYS: &'static str;

    fn values(&self) -> String;

    fn csv(&self) -> String {
        format!("{}\n{}", Self::KEYS, self.values())
    }

    fn write_csv(&self, file: &str) -> std::io::Result<()> {
        std::fs::write(file, self.csv())
    }
}

With this definition, a CSV is seperated into two parts. There are the keys(or headers) and the values. We should store the keys as a constant. They won't need to change, or be programatically generated. The values will have to be generated as a String. We can combine the keys and values to create a full spreadsheet.  

Now we can implement this trait for `DisplayValues`.

In [9]:
impl ToCSV for DisplayValues {
    const KEYS: &'static str = "Id,Value,Parity,Difference,Mod 4";

    fn values(&self) -> String {
        format!(
            "{},{},{:?},{},{}",
            self.id, self.value, self.parity, self.difference, self.mod_4
        )
    }
}

While we're at it, we might as well implement `ToCSV` for every `Vec<T>` where `T` already implements `ToCSV`. With that implementation, you have an incredibly simple way to create a CSV from pretty much any struct.

In [10]:
impl<T: ToCSV> ToCSV for Vec<T> {
    const KEYS: &'static str = T::KEYS;

    fn values(&self) -> String {
        self.into_iter()
            .map(|x| x.values() + "\n")
            .fold(String::new(), |acc, s| acc + &s)
    }
}

With the `ToCSV` trait implemented for both types, we can now turn any vector of `DisplayValues` into a CSV string.

In [11]:
let csv = display_values.csv();
println!("{csv}");

We can also do it all in one step like this:

In [12]:
let csv: String = Primes::new(10).to_dv().csv();
println!("{csv}");

Id,Value,Parity,Difference,Mod 4
0,2,Even,0,2
1,3,Odd,1,3
3,7,Odd,2,3
2,5,Odd,2,1
4,11,Odd,4,3
5,13,Odd,2,1
6,17,Odd,4,1
7,19,Odd,2,3
8,23,Odd,4,3
9,29,Odd,6,1



All of our data is now in CSV format. It isn't very usable like this, so our final step will be to output the `csv` String to a file. Lukily, we've already implemented a method in `ToCSV` which does just that. Any type that implements `ToCSV` will easily be able to write out to a csv file.

In [13]:
Primes::new(100).to_dv().write_csv("out.csv");